In [1]:
import cherrypy
import json
import redis
import datetime
'''
REDIS_HOST = 'redis-13201.c300.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 13201
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'acD7xZwf4t52f4Y6E5TSUbeewaXculrM'
'''
REDIS_HOST = 'redis-10188.c135.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 10188
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'A7WdgwW3KBkCkq1qQxHnTt9aF4lfR6XY'
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {'status': 'online'}
        response = json.dumps(response_dict)
        return response

class DeviceList(object):
    exposed = True

    def GET(self, *path, **query):
        blt = query.get('blt', None)
        plugged = query.get('plugged', None)

        # Validate query parameters
        if blt is not None:
                blt = int(blt)
                if not 0 <= blt <= 100:
                    raise ValueError
        if plugged is not None:
                plugged = int(plugged)
                if plugged not in [0, 1]:
                    raise ValueError

        # Retrieve and filter devices from Redis
        device_keys = redis_client.keys("*:battery")
        mac = []

        for key in device_keys:
            new_key = key.decode()
            mac_address = new_key.split(':')[0]
            try:
                power_device = redis_client.ts().range(f'{mac_address}:power', "-", "+")
                battery_device= redis_client.ts().range(f'{mac_address}:battery', "-", "+")
                
            except redis.exceptions.ResponseError:
                continue

            battery_percentage = [i[1] for i in battery_device]
            power_statuses = [j[1] for j in power_device]

            if ((blt is None or max(battery_percentage) <= blt) and
                    (plugged is None or plugged in power_statuses)):
                mac.append(mac_address)
        mac = list(set(mac))
        response = {'mac_addres': mac}
        return json.dumps(response)

class DeviceBatteryStatus(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) < 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]

        start_date = query.get('start_date', None)
        end_date = query.get('end_date', None)

        if not mac_address:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')

        try:
            if start_date:
                start_date = datetime.datetime.fromisoformat(start_date)
            if end_date:
                end_date = datetime.datetime.fromisoformat(end_date)
                if start_date and start_date >= end_date:
                    raise ValueError
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: invalid date format or range')

        power_device = redis_client.ts().range(f'{mac_address}:power', "-", "+")
        battery_device= redis_client.ts().range(f'{mac_address}:battery', "-", "+")

        timestamp = []
        battery_percentage = []
        power_statuses = []

        for i in range(min(len(battery_device), len(power_device))):
            timestamp_battery, battery_level = battery_device[i]
            timestamp_power, power_status = power_device[i]

            timestamp_seconds = timestamp_battery / 1000
            date_obj = datetime.datetime.fromtimestamp(timestamp_seconds)
            if start_date <= date_obj <= end_date:
                timestamp.append(timestamp_battery)
                battery_percentage.append(battery_level)
                power_statuses.append(power_status)


        response = {
            'mac_address': mac_address,
            'timestamps': timestamp,
            'battery_levels': battery_percentage,
            'power_plugged': power_statuses
        }

        return json.dumps(response)
    def DELETE(self, *path, **query):
        if len(path) < 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        print(mac_address)

        if not mac_address:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')

        device_key = f'{mac_address}:battery'

        exists = redis_client.exists(device_key)
        if not exists:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        redis_client.delete(device_key)

        return json.dumps({'status': 'success', 'message': 'Timeseries data deleted'})

if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(DeviceList(), '/devices', conf)
    cherrypy.tree.mount(DeviceBatteryStatus(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[01/Feb/2024:14:19:57] ENGINE Bus STARTING
[01/Feb/2024:14:19:57] ENGINE Started monitor thread 'Autoreloader'.
[01/Feb/2024:14:19:57] ENGINE Serving on http://0.0.0.0:8080
[01/Feb/2024:14:19:57] ENGINE Bus STARTED
Redis Connected: True
127.0.0.1 - - [01/Feb/2024:14:20:12] "GET /devices HTTP/1.1" 200 88 "" "python-requests/2.28.1"
127.0.0.1 - - [01/Feb/2024:14:20:13] "GET /device/0xd0b36afd8e6c?start_date=2024-01-15+00%3A00%3A00&end_date=2024-02-03+00%3A00%3A00 HTTP/1.1" 200 1128 "" "python-requests/2.28.1"
127.0.0.1 - - [01/Feb/2024:14:20:13] "GET /device/0xec63d75a2354?start_date=2024-01-15+00%3A00%3A00&end_date=2024-02-03+00%3A00%3A00 HTTP/1.1" 200 114 "" "python-requests/2.28.1"
127.0.0.1 - - [01/Feb/2024:14:20:13] "GET /device/0x3822e23d5ead?start_date=2024-01-15+00%3A00%3A00&end_date=2024-02-03+00%3A00%3A00 HTTP/1.1" 200 2688 "" "python-requests/2.28.1"
127.0.0.1 - - [01/Feb/2024:14:20:14] "GET /device/0xf14d2b988308?start_date=2024-01-15+00%3A00%3A00&end_date=2024-02-03+00%3A00%

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f4c1e8e7-3e9e-4aba-a439-c0974b52f6c7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>